In [1]:
# Google Drive Mount

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/Song-Joo-Young/lattice.git Lattice_repo

fatal: destination path 'Lattice_repo' already exists and is not an empty directory.


In [3]:
%cd Lattice_repo

/content/Lattice_repo


In [4]:
! pip install -r requirements.txt

  Using cached torch-1.12.1-cp310-cp310-manylinux1_x86_64.whl (776.3 MB)
  Using cached transformers-4.4.0-py3-none-any.whl (2.1 MB)
  Using cached datasets-2.4.0-py3-none-any.whl (365 kB)
  Using cached sentencepiece-0.1.97-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached jsonlines-3.1.0-py3-none-any.whl (8.6 kB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Using cached multiproces

In [6]:
!pip install datasets==2.4.0
!pip install sentencepiece==0.1.97
!pip install sacrebleu==2.2.1
!pip install jsonlines==3.1.0
!pip install absl-py==1.2.0

  Using cached datasets-2.4.0-py3-none-any.whl (365 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Using cached multiprocess-0.70.15-py310-none-any.whl (134 kB)
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)
  Using cached multiprocess-0.70.13-py310-none-any.whl (133 kB)
  Using cached sentencepiece-0.1.97-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99
  Using cached jsonlines-3.1.0-py3-none-any.whl (8.6 kB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Attempting uninstall: ab

In [7]:
import json
import os

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import T5Tokenizer

# from preprocess_utils import get_highlighted_subtable, linearize_subtable

In [7]:
 ! wget https://storage.googleapis.com/totto-public/totto_data.zip
 ! unzip totto_data.zip

--2024-02-11 14:37:09--  https://storage.googleapis.com/totto-public/totto_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.136.207, 142.250.148.207, 209.85.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.136.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187724372 (179M) [application/zip]
Saving to: ‘totto_data.zip’

totto_data.zip      100%[===================>] 179.03M   156MB/s    in 1.2s    

2024-02-11 14:37:10 (156 MB/s) - ‘totto_data.zip’ saved [187724372/187724372]

Archive:  totto_data.zip
  inflating: totto_data/totto_dev_data.jsonl  
  inflating: totto_data/totto_train_data.jsonl  
  inflating: totto_data/unlabeled_totto_test_data.jsonl  


In [8]:
! python preprocess/preprocess_data.py --input_path="totto_data/totto_dev_data.jsonl" --output_path="totto_data/dev_linearized.jsonl"
! python preprocess/json_to_csv.py -i totto_data/dev_linearized.jsonl -o totto_data/dev.csv

! python preprocess/preprocess_data.py --input_path="totto_data/totto_train_data.jsonl" --output_path="totto_data/train_linearized.jsonl"
! python preprocess/json_to_csv.py -i totto_data/train_linearized.jsonl -o totto_data/train.csv

7700it [00:29, 262.35it/s]
Num examples processed: 7700
7700it [00:04, 1727.87it/s]
120761it [07:48, 257.67it/s]
Num examples processed: 120761
120761it [00:51, 2325.95it/s]


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print("Current GPU Index:", torch.cuda.current_device())
    print("Current GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))

In [11]:
import shutil

shutil.move("/content/Lattice_repo", "/content/drive/MyDrive/")

'/content/drive/MyDrive/Lattice_repo'

In [13]:
import os

# 작업 디렉토리 변경
os.chdir("/content/drive/MyDrive/Lattice_repo")

# 현재 작업 디렉토리 확인
print("현재 작업 디렉토리:", os.getcwd())

현재 작업 디렉토리: /content/drive/MyDrive/Lattice_repo


In [15]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.5 MB/s eta 0:00:00


In [16]:
'''
CUDA_VISIBLE_DEVICES=0 python train.py \
--model_name_or_path t5-base \
--do_train \
--do_eval \
--do_predict \
--train_file totto_data/train.csv \
--validation_file totto_data/dev.csv \
--test_file totto_data/dev.csv \
--output_dir outputs/ \
--per_device_train_batch_size 8 \
--gradient_accumulation_steps 1 \
--per_device_eval_batch_size 32 \
--group_by_length \
--max_source_length 512 \
--max_target_length 128 \
--val_max_target_length 128 \
--num_beams 4 \
--max_steps 150000 \
--learning_rate 2e-4 \
--lr_scheduler_type constant \
--evaluation_strategy steps \
--eval_steps 5000 \
--logging_steps  5000 \
--save_steps 5000 \
--metric_for_best_model bleu \
--load_best_model_at_end \
--dataloader_num_workers 8 \
--overwrite_cache \
--overwrite_output_dir \
--predict_with_generate
'''
!bash run.sh

2024-02-11 14:59:44.385006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 14:59:44.385076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 14:59:44.386926: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 14:59:45.872606: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
02/11/2024 14:59:47 - WARNING - __main__ -   Process rank: 0, device: cpu, n_gpu: 0distributed training: True, 16-bits training: False
02/11/2024 14:59:47 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=